# Vizualiziranje netrenirane mreže

Ova bilježnica je tu jer su mi neki rezultati, well, bili _čudni_, odnosno činilo se da neuroni pokazuju više pravilnosti u checkpointu _prije treniranja_. Zato ovdje ide svježe-učitana mreža po skalpel, s težinama određenima s kojim već defaultnim 

In this notebook I'll just do some interpretability on a newly initialized network. Which, well, shouldn't really be possible, except insofar CNNs have some hellishly good inductive biases(???).

## Definicije i importovi

In [ ]:
pip install torch-lucent

In [ ]:
from lucent.optvis.transform import pad, jitter, random_rotate, random_scale
from lucent.optvis import render, param, transform, objectives
from tqdm import tqdm

In [ ]:
import torch

from lucent.optvis import render, param, transform, objectives

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.AvgPool2d(4, 4)
        
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.AvgPool2d(4, 4)
        
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.AvgPool2d(4, 4)
        
        self.conv3_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.AvgPool2d(2 , 2)
        
        self.fc = nn.Linear(324, 4)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv1_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv2_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv3_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x


model = Net()
model = model.to(device)

In [ ]:
model.eval()

## Sloj po sloj!

#### Prvi (1.) (konvolucijski)

In [ ]:
n_row = 3
n_col = 3
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
  img = render.render_vis(model, f"conv3:{ix}", progress=False, show_image=False)[0]
  #img = np.transpose(img, [0, 3, 2, 1])
  img = np.reshape(img, (128, 128, 3))
  ax.imshow(img)
  ax.set_xticklabels([])
  ax.set_yticklabels([])
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

#### Drugi (2.)

In [ ]:
n_row = 4
n_col = 4
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
  img = render.render_vis(model, f"conv2:{ix}", progress=False, show_image=False)[0]
  #img = np.transpose(img, [0, 3, 2, 1])
  img = np.reshape(img, (128, 128, 3))
  ax.imshow(img)
  ax.set_xticklabels([])
  ax.set_yticklabels([])
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

#### Treći (3.)

In [ ]:
n_row = 5
n_col = 5
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
  img = render.render_vis(model, f"conv3:{ix}", progress=False, show_image=False)[0]
  #img = np.transpose(img, [0, 3, 2, 1])
  img = np.reshape(img, (128, 128, 3))
  ax.imshow(img)
  ax.set_xticklabels([])
  ax.set_yticklabels([])
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

#### Četvrti (4.)

In [ ]:
n_row = 6
n_col = 6
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
  img = render.render_vis(model, f"conv4:{ix}", progress=False, show_image=False)[0]
  #img = np.transpose(img, [0, 3, 2, 1])
  img = np.reshape(img, (128, 128, 3))
  ax.imshow(img)
  ax.set_xticklabels([])
  ax.set_yticklabels([])
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

# Captum (to the rescue?)

In [ ]:
!git clone https://github.com/pytorch/captum
%cd captum
!git checkout "optim-wip"
!pip3 install -e .
import sys
sys.path.append('/content/captum')
%cd ..


In [ ]:
import captum.optim as optimviz
import torchvision

In [ ]:
from typing import Callable, Iterable, Optional

In [ ]:
def vis_neuron_large(
    model: torch.nn.Module, target: torch.nn.Module, channel: int
) -> None:
    image = optimviz.images.NaturalImage((640, 640)).to(device)
    transforms = torch.nn.Sequential(
        torch.nn.ReflectionPad2d(2),
        optimviz.transforms.RandomSpatialJitter(8),
        optimviz.transforms.RandomScale(scale=(1.15, 0.85, 1, 0.95, 1.05)),
        torchvision.transforms.RandomRotation(degrees=(-15, 15)),
        optimviz.transforms.RandomSpatialJitter(16),
        optimviz.transforms.CenterCrop((640, 640)),
    )
    loss_fn = optimviz.loss.NeuronActivation(target, channel)
    obj = optimviz.InputOptimization(model, loss_fn, image, transforms)
    history = obj.optimize(optimviz.optimization.n_steps(128, False))
    return image()

In [ ]:
n_row = 3
n_col = 3
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
    img = vis_neuron_large(model, model.conv1, ix)
    img = img.permute(0, 2, 3, 1)
    with torch.no_grad():
        img = img.cpu().numpy()
    img = img.reshape((640,640,3))
    ax.imshow(img)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.margins(x=0, y=0, tight=True)

    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
render.render_vis?

In [ ]:
n_row = 4
n_col = 4
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
    img = vis_neuron_large(model, model.conv2, ix)
    img = img.permute(0, 2, 3, 1)
    with torch.no_grad():
        img = img.cpu().numpy()
    img = img.reshape((640,640,3))
    ax.imshow(img)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.margins(x=0, y=0, tight=True)

    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
n_row = 5
n_col = 5
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
    img = vis_neuron_large(model, model.conv3, ix)
    img = img.permute(0, 2, 3, 1)
    with torch.no_grad():
        img = img.cpu().numpy()
    img = img.reshape((640,640,3))
    ax.imshow(img)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.margins(x=0, y=0, tight=True)

    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
n_row = 6
n_col = 6
_, axs = plt.subplots(n_row, n_col, figsize=(14, 14))
axs = axs.flatten()
for ix, ax in tqdm(zip(range(n_row*n_col), axs)):
    img = vis_neuron_large(model, model.conv4, ix)
    img = img.permute(0, 2, 3, 1)
    with torch.no_grad():
        img = img.cpu().numpy()
    img = img.reshape((640,640,3))
    ax.imshow(img)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.margins(x=0, y=0, tight=True)

    
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()